# KFServing deploying demo

## Deploy Chassis built image to KFServing

This demo will show you how we can deploy an image built using `Chassis` to an existing installation of KFServing.

This is assuming that you have already containerizer your model using `Chassis` service and the image can be downloaded from a public Docker Hub account.

## Prepare KFServing deployment

First we define the image tag that has been pushed to Docker Hub.

Example: `carmilso/chassisml-sklearn-demo:latest`.

In [1]:
image_tag = input('Image tag: ')

import os
os.environ['IMAGE_TAG'] = image_tag

Image tag: carmilso/chassisml-sklearn-demo:latest


Now we create the KFServing deployment definition. We will use the image built with Chassis.

In [2]:
%%bash

cat <<- EOF > kfserving_v1.yaml
apiVersion: "serving.kubeflow.org/v1beta1"
kind: "InferenceService"
metadata:
  name: "chassis-kfserving-v1"
spec:
  predictor:
    containers:
    - image: "$IMAGE_TAG"
      name: "chassis-kfserving-v1-container"
      env:
        - name: INTERFACE
          value: kfserving
        - name: HTTP_PORT
          value: "8080"
        - name: PROTOCOL
          value: v1
        - name: MODEL_NAME
          value: digits
      ports:
        - containerPort: 8080
          protocol: TCP
EOF

## Deploy the image

Now, we are going to apply that configuration file so our image is deployed in the KFServing installation.

In [ ]:
# Run all these commands on testfaster SSH client.
kubectl cp $(kubectl get pod -l app=combinator-jupyter -o custom-columns=":metadata.name" | tr -d '\n'):kfserving_v1.yaml kfserving_v1.yaml
kubectl apply -f kfserving_v1.yaml

export INGRESS_HOST=$(minikube ip)
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
export SERVICE_NAME=chassis-kfserving-v1
export MODEL_NAME=digits


Once we have defined the required variables we need to wait for the pod to be up and running. Once this is finished we can get the service hostname.

In [ ]:
# Run all these commands on testfaster SSH client.
# Wait until the pod is created and run the following command.
export SERVICE_HOSTNAME=$(kubectl get inferenceservice ${SERVICE_NAME} -o jsonpath='{.status.url}' | cut -d "/" -f 3)


We copy the inputs file that contains the data that we want to predict.

In [ ]:
# Run all these commands on testfaster SSH client.
kubectl cp $(kubectl get pod -l app=combinator-jupyter -o custom-columns=":metadata.name" | tr -d '\n'):containerizer/service/flavours/mlflow/interfaces/kfserving/inputsv1.json inputsv1.json


## Predict data

Now you can just make a request to predict some data:

In [ ]:
# Run all these commands on testfaster SSH client.
curl -H "Host: ${SERVICE_HOSTNAME}" "http://${INGRESS_HOST}:${INGRESS_PORT}/v1/models/${MODEL_NAME}:predict" -d@inputsv1.json | jq
